In [16]:
import pandas as pd
import re as lib
import os
from google.colab import drive
from datetime import datetime

In [17]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
# Especifica la ruta completa del archivo
ruta_archivo = '/content/drive/MyDrive/tw_source.csv'

# Carga el archivo CSV en un DataFrame
df = pd.read_csv(ruta_archivo, encoding='latin1')

# Muestra las primeras 5 filas del DataFrame
print(df.head())

   0  1467810369  Mon Apr 06 22:19:45 PDT 2009  NO_QUERY _TheSpecialOne_  \
0  0  1467810672  Mon Apr 06 22:19:49 PDT 2009  NO_QUERY   scotthamilton   
1  0  1467810917  Mon Apr 06 22:19:53 PDT 2009  NO_QUERY        mattycus   
2  0  1467811184  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY         ElleCTF   
3  0  1467811193  Mon Apr 06 22:19:57 PDT 2009  NO_QUERY          Karoli   
4  0  1467811372  Mon Apr 06 22:20:00 PDT 2009  NO_QUERY        joy_wolf   

  @switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D  
0  is upset that he can't update his Facebook by ...                                                                   
1  @Kenichan I dived many times for the ball. Man...                                                                   
2    my whole body feels itchy and like its on fire                                                                    
3  @nationwideclass no, it's not behaving at all....           

In [23]:
columnas = ['Code', 'Longitud', 'Date', 'Query', 'User', 'Content']
df.columns = columnas
df

,Code,Longitud,Date,Query,User,Content
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
1599994,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599995,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599996,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599997,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [24]:
import re
from collections import Counter

# Extraer todos los usuarios mencionados con una expresión regular
df['mentions'] = df['Content'].apply(lambda x: re.findall(r'@\w+', str(x)))

# Contar todas las menciones
all_mentions = [mention for mentions_list in df['mentions'] for mention in mentions_list]
user_count = Counter(all_mentions)

# Encontrar los 3 usuarios más populares
top_users = user_count.most_common(3)
top_users


[('@mileycyrus', 4500), ('@tommcfly', 3887), ('@ddlovato', 3467)]

In [32]:
# Crear un corpus para cada usuario
corpus = {}

for user, _ in top_users:
    user_tweets = df[df['User'].str.contains(user)]
    corpus[user] = {
        'Content': user_tweets['User'].tolist(),
        'Metadata': {
            'Code': user_tweets['Code'].tolist(),
            'Date': user_tweets['Date'].tolist(),
            'Longitud': user_tweets['User'].apply(len).tolist(),
        }
    }

# Verificamos el corpus de uno de los usuarios
corpus[top_users[0][0]]


{'Content': [], 'Metadata': {'Code': [], 'Date': [], 'Longitud': []}}

In [33]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess_text(text):
    # Convertir a minúsculas, eliminar signos de puntuación y stopwords
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Stemming y lematización
    words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    return ' '.join(words)

# Preprocesar los tweets de cada usuario
for user in corpus:
    corpus[user]['Processed_Content'] = [preprocess_text(tweet) for tweet in corpus[user]['Content']]

# Verificar el contenido procesado de uno de los usuarios
corpus[top_users[0][0]]['Processed_Content'][:5]


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


[]

In [35]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

In [43]:
!pip install wordcloud matplotlib

In [44]:
def generate_wordcloud(text, user):
    wordcloud = WordCloud(width=800, height=400, background_color='white').generate(' '.join(text))
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title(f'Wordcloud for {user}')
    plt.show()


In [45]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
import string

nltk.download('stopwords')
nltk.download('wordnet')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

def preprocess_text(text):
    # Convertir a minúsculas, eliminar signos de puntuación y stopwords
    text = text.lower()
    text = ''.join([char for char in text if char not in string.punctuation])
    words = text.split()
    words = [word for word in words if word not in stop_words]

    # Stemming y lematización
    words = [stemmer.stem(word) for word in words]
    words = [lemmatizer.lemmatize(word) for word in words]

    # Return empty string if no words left after preprocessing
    # to prevent the ValueError
    if not words:
        return ''
    return ' '.join(words)

# Preprocesar los tweets de cada usuario
for user in corpus:
    corpus[user]['Processed_Content'] = [preprocess_text(tweet) for tweet in corpus[user]['Content']]

# Verificar el contenido procesado de uno de los usuarios
corpus[top_users[0][0]]['Processed_Content'][:5]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


[]